In [1]:
import sqlalchemy
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import shap

from datetime import datetime
import pytz
from tqdm import tqdm
import pickle
import random


In [2]:
df = pd.read_csv('Bulk_Carrier_202001_202006.csv')

In [3]:
df['TIME_STAMP'] = pd.to_datetime(df['TIME_STAMP']).dt.tz_localize(None)

# Get last period of data

In [4]:
# by interval
interval_day = 180
# dff = df.iloc[-interval_day:]

# full dataset
dff = df.copy()

# Preprocessing

In [5]:
me1 = [i for i in dff.columns if i.startswith('ME1')]
me1 = [i for i in me1 if not i.endswith('ECC')]

to_exclude = ['ME1_FO_FLOW_HOUR_INLET',
            'ME1_FO_DENSITY_INLET',
            'ME1_FO_TEMP_INLET',
            'ME1_FO_TOTALIZER_INLET']


me1 = [i for i in me1 if i not in to_exclude]
me2 = ['SPEED_VG','lon','lat','TIME_STAMP']


df2=pd.concat([dff[me1], dff[me2]], axis=1)
df2 = df2.loc[:,~df2 .columns.duplicated()]

# Filter Value
df3=df2[(df2['ME1_RPM']>0) & (df2['ME1_SCAV_AIR_PRESS']>0) & (df2['ME1_LO_INLET_PRESS']>0) & (df2['ME1_FOC']>0)]
df3=df3[(df3['ME1_RPM']>df3['ME1_RPM'].describe()['75%']) & (df3['SPEED_VG'] > 14)]

# Aggregating Cylinders Variable (Averaging)
pco=[i for i in df3.columns if 'PCO' in i]
cfw=[i for i in df3.columns if 'CFW' in i]
exh=[i for i in df3.columns if 'EXH' in i]
df3['ME1_CYL_PCO_OUTLET_TEMP']=df3[pco].mean(axis=1)
df3['ME1_CYL_CFW_OUTLET_TEMP']=df3[cfw].mean(axis=1)
df3['ME1_CYL_EXH_GAS_OUTLET_TEMP']=df3[exh].mean(axis=1)
df3 = df3.drop(pco+cfw+exh,axis=1)

df4 = df3.drop(['SPEED_VG','lon','lat','TIME_STAMP'],axis=1).copy()

# Isolation Forest

In [6]:
iso = IsolationForest(random_state=4)
iso.fit(df4)
iso_score=iso.score_samples(df4)

# SHAP 

In [7]:
# SHAP Explainer
explainer = shap.TreeExplainer(iso,data=df4)
#Calculate shap with shap_values method
shap_values = explainer.shap_values(df4,check_additivity=True)
dfShap = pd.DataFrame(shap_values,columns=df4.columns,index=df4.index)

dfShap_temp = dfShap[:].copy()
dfShap_temp['ISO_SCORE'] = iso_score

 98%|===================| 2175/2222 [00:18<00:00]        

# Automatic Clustering

In [8]:
MIN_CLUSTER = 2
MAX_CLUSTER = 8

In [9]:
fixed_th = np.quantile(dfShap_temp['ISO_SCORE'],0.075)
dfShap_temp1 = dfShap_temp[dfShap_temp['ISO_SCORE'] <= fixed_th].drop(['ISO_SCORE'],axis=1)
dfShap_temp2 = dfShap_temp[dfShap_temp['ISO_SCORE'] > fixed_th].drop(['ISO_SCORE'],axis=1)
        

def auto_cluster(df_shap, min_cluster, max_cluster):
    num_class = 0
    sil_score = 0

    if len(df_shap) >= 3 :
        for num_cluss in tqdm(range(min_cluster,max_cluster+1)):
            kms_mod = KMeans(n_clusters=num_cluss,random_state=42)
            labels_test = kms_mod.fit_predict(df_shap)
            centroids_test = kms_mod.cluster_centers_
            silhouette_avg = silhouette_score(df_shap, labels_test)
            if silhouette_avg > sil_score :
                sil_score = silhouette_avg
                num_class = num_cluss
            else :
                pass 
    else :
        num_class = 1   

    # ---- Start Model with Best Num Clus
    kms_mod = KMeans(n_clusters=num_class,random_state=42)
    labels_test = kms_mod.fit_predict(df_shap)
    centroids_test = kms_mod.cluster_centers_

    return labels_test, centroids_test, kms_mod

labels_cluster, centroids_cluster, model_cluster =  auto_cluster(dfShap_temp1, MIN_CLUSTER, MAX_CLUSTER)

dfShap_temp1['CLUSTER'] = labels_cluster
dfShap_temp2['CLUSTER'] = np.nan

dfShap = pd.concat([dfShap_temp1, dfShap_temp2])

100%|██████████| 7/7 [00:36<00:00,  5.21s/it]


# Datamart

In [10]:
# datamart creation
datamart = df3.join(dfShap, rsuffix='_SHAP') 
datamart['ISO_SCORE'] = iso_score
datamart = datamart[datamart.columns.difference(df2.columns)]


datamart = df2.join(datamart)
datamart['INGESTION_DT'] = dff['TIME_STAMP']

In [11]:
datamart.head()

,ME1_RPM,ME1_RPM_SHAFT,ME1_SHAFT_POWER,ME1_SHAFT_TORQUE,ME1_SHAFT_THRUST,ME1_FO_INLET_PRESS,ME1_FO_INLET_TEMP,ME1_JCW_INLET_TEMP,ME1_JCW_INLET_PRESS,ME1_CYL1_CFW_OUTLET_TEMP,...,ME1_SHAFT_THRUST_SHAP,ME1_SHAFT_TORQUE_SHAP,ME1_TC1_LO_INLET_PRESS_SHAP,ME1_TC1_LO_OUTLET_TEMP_SHAP,ME1_TC1_RPM_SHAP,ME1_TC2_LO_INLET_PRESS_SHAP,ME1_TC2_LO_OUTLET_TEMP_SHAP,ME1_TC2_RPM_SHAP,ME1_THRUST_PAD_TEMP_SHAP,INGESTION_DT
0,51.060,50.9,0.0,0.0,2220.0,7.686,134.763,78.115,4.352,119.036,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 00:00:00
1,50.970,50.9,0.0,0.0,2220.0,7.687,133.605,77.957,4.340,119.012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 00:10:00
2,51.120,50.9,0.0,0.0,2220.0,7.697,135.074,78.249,4.356,119.076,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 00:20:00
3,54.042,50.9,0.0,0.0,2220.0,7.637,135.089,77.434,4.355,118.808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 00:30:00
4,56.376,50.9,0.0,0.0,2220.0,7.593,135.735,77.107,4.355,117.144,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01 00:40:00
